* `/srv/local2/abel/event-representation-for-causal/data/cgw/temporal/causal-cmd`

In [58]:
import jsonlines
import os
import random
from collections import Counter

In [2]:
project_folder = os.environ["EVENT_HOME"]

Disease Outbreak

Irrelevant temporal extractions:
*   `cure` before `spread`: "Myths like 'you can avoid AIDS by sleeping only with fat women,' and 'you can `cure` yourself by sleeping with a virgin,' are rife." -> "They would `spread` you all over."
* `study` before `spread`: "Study after study is showing that the efforts to maintain safer sex practices are breaking down, particularly among white men." -> "Through poetry and hip hop, they `spread` a gay-positive message for African Americans: There's nothing wrong with brothers loving each other."


The temporal extractions are hard to verify because they do not contain references back to the original article. Or, if they do exist, it is extremely difficult to find. And, the relations do not keep their POS tags.

An edge (u, v) appers there if LOME output "v before u" at least three times more frequently than "u bevore v", and both u and v appeared at least 30 times


In [47]:
articles = {}
with jsonlines.open(f"{project_folder}/results/disease_outbreak/nyt_en_disease_outbreak.jsonl") as reader:
    for obj in reader.iter():
        articles[obj['id']] = obj

with jsonlines.open(f"{project_folder}/results/disease_outbreak/temporal_relations.jsonl") as reader:
    for obj in reader.iter():
        doc_id = obj['filename'].split("/")[-1][:-5]
        articles[doc_id]['relations'] = obj['relations']
        present_verbs = set()
        for i in obj['relations']:
            present_verbs.add(i[1])
            present_verbs.add(i[2])
        articles[doc_id]['verbs'] = present_verbs


Collect temporal relation stats

In [61]:
relation_counts = Counter()
for doc_id, info in articles.items():
    if not info.get('relations'):
        continue
    relation_counts.update([i[0] for i in info['relations']])

for rel, c in relation_counts.items():
    print(f"{rel}: {c:,}")


before: 18,526
after: 1,081


Find articles based on verbs

In [57]:
verbs = ["infect", "evolve"]

def is_relevant(obj):
    if 'verbs' not in obj:
        return False
    if verbs[0] in obj['verbs'] and verbs[1] in obj['verbs']:
        return True
    return False

for doc_id, info in articles.items():
    if is_relevant(info):
        print(f"Relations: {info['relations']}\n")
        text = "\n".join(info['text'])
        print(f"{text}")
        print("---------------------------------")

Relations: [['before', 'spread', 'evolve'], ['before', 'spread', 'spread'], ['before', 'spread', 'spread'], ['before', 'spread', 'infect'], ['before', 'spread', 'spread'], ['before', 'spread', 'test'], ['before', 'evolve', 'spread'], ['before', 'evolve', 'spread'], ['before', 'evolve', 'infect'], ['before', 'evolve', 'spread'], ['before', 'evolve', 'test'], ['before', 'spread', 'spread'], ['before', 'spread', 'infect'], ['before', 'spread', 'spread'], ['before', 'spread', 'test'], ['before', 'spread', 'infect'], ['before', 'spread', 'spread'], ['before', 'spread', 'test'], ['before', 'infect', 'spread'], ['before', 'infect', 'test'], ['before', 'spread', 'test']]

SAN FRANCISCO, Calif.--The mysterious, flulike illness from southern China known as severe acute respiratory syndrome, or SARS, has public health officials around the world on high alert, and international travelers are trying to decide whether to cancel trips to Asia or even parts of Canada, where a cluster of cases arose in

Examine random article

In [ ]:
article_id = random.choice(list(articles.keys()))

articles[article_id]